Deliverables:
* Predictions for which children will or not receive all vaccines by 6 months on a test set of data (generated by you)
* All notebooks/scripts used to create predictions
    * Data exploration
    * A defined baseline
    * Predictive model, can you beat the baseline?
* A description of next steps:
    * If you had more time to work on this problem, what other data sources or methods would you like to add that you think would improve the predictions, and why?
    * How can your results help health facilities allocate health workers?
